Fine-tuning and Deployment:

- Fine-tune the best performing model on the entire dataset to improve its performance further.
- Save the trained model and deploy it on a server or in the cloud to classify new histopathelogy images.

In [1]:
import sys
sys.path.append('../../src')

import preprocessing
